In [0]:
# Autor: Jairo Cifuentes
# Fecha: 06/08/2025
# Descripción: Carga de datos dn_user en Bronze

In [0]:
dbutils.widgets.text("configID","1")
configID = dbutils.widgets.get("configID")


In [0]:
configuration =  spark.sql(f"select cliente,fuente,ruta_archivo,formato,tabla_destino,frecuencia,activo,\
                             ultima_ejecucion,schema from hive_metastore.bronze.tbl_config_ingesta \
                             where id ='{configID}'")

In [0]:
ruta_archivo = configuration.collect()[0]["ruta_archivo"]
fuente = configuration.collect()[0]["fuente"]
formato = configuration.collect()[0]["formato"]
tabla_destino = configuration.collect()[0]["tabla_destino"]
frecuencia = configuration.collect()[0]["frecuencia"]
schema = configuration.collect()[0]["schema"]

In [0]:
display(configuration)

In [0]:
%sql
/*update 
hive_metastore.bronze.tbl_config_ingesta
set ruta_archivo ='{\"scope\":\"secret-storeview\", \"username\":\"username-keycloak-db\", \"password\":\"password-keycloak-db\", \"database\":\"keycloak\",\"hostname\": \"psql-dn-keycloak-restore-2.postgres.database.azure.com\", \"port\":5432, \"dbtable\":\"dn_user\"  }'*/

In [0]:
import json

config = json.loads(ruta_archivo)

In [0]:
config["scope"]

In [0]:
#Configuración de conexión JDBC
JDBC_CONFIG = {
    "hostname": config["hostname"],
    "port": config["port"],
    "database": config["database"],
    "username": dbutils.secrets.get(scope=config["scope"], key=config["username"]),
    "password": dbutils.secrets.get(scope=config["scope"], key=config["password"]),
    "driver": "org.postgresql.Driver"
}

jdbcUrl = f"jdbc:postgresql://{JDBC_CONFIG['hostname']}:{JDBC_CONFIG['port']}/{JDBC_CONFIG['database']}?sslmode=require"
connectionProperties = {
    "user": JDBC_CONFIG["username"],
    "password": JDBC_CONFIG["password"],
    "driver": JDBC_CONFIG["driver"]
}

In [0]:
def encode_schema(schema):
    """
    Encodes a StructType schema into a string that can be passed as a parameter.
    
    Args:
        schema (StructType): The schema to encode.
    
    Returns:
        str: A string representing the encoded schema.
    """
    schema_str = ""
    for field in schema:
        field_type = field.dataType.simpleString()
        nullable = field.nullable
        schema_str += f"{field.name},{field_type},{nullable}\n"
    return schema_str.strip()


In [0]:
from pyspark.sql.types import (
    StructType, StructField, StringType, IntegerType, LongType, DecimalType,
    DateType, DoubleType, FloatType, BooleanType, ShortType, BinaryType, TimestampType
)
def decode_schema(schema_str):
    """
    Decodes a string into a StructType schema.
    
    Args:
        schema_str (str): The string representing the encoded schema.
    
    Returns:
        StructType: The decoded schema.
    
    Raises:
        ValueError: If there's an error processing a schema line.
    """
    fields = []
    lines = schema_str.strip().split('\n')
    for line in lines:
        try:
            field_name, rest = line.split(',', 1)
            
            if rest.startswith('decimal('):
                decimal_part, nullable = rest.rsplit(',', 1)
                field_type = decimal_part
                nullable = nullable.lower() == 'true'
                precision, scale = map(int, decimal_part[8:-1].split(','))
                data_type = DecimalType(precision, scale)
            else:
                field_type, nullable = rest.rsplit(',', 1)
                nullable = nullable.lower() == 'true'
                
                if field_type == "string":
                    data_type = StringType()
                elif field_type in ["int", "integer"]:
                    data_type = IntegerType()
                elif field_type in ["long", "bigint"]:
                    data_type = LongType()
                elif field_type == "double":
                    data_type = DoubleType()
                elif field_type == "float":
                    data_type = FloatType()
                elif field_type == "boolean":
                    data_type = BooleanType()
                elif field_type == "short":
                    data_type = ShortType()
                elif field_type == "binary":
                    data_type = BinaryType()
                elif field_type == "timestamp":
                    data_type = TimestampType()
                elif field_type == "date":
                    data_type = DateType()
                else:
                    raise ValueError(f"Tipo de dato no soportado: {field_type}")
            
            fields.append(StructField(field_name, data_type, nullable))
        except Exception as e:
            raise ValueError(f"Error al procesar la línea '{line}': {str(e)}")
    
    return StructType(fields)


In [0]:
#Se importa los tipos de datos necesarios para definir el esquema del DataFrame
from pyspark.sql.types import StructType, StructField, StringType, BooleanType

# Controlar los tipos de datos y evitar inferencias automáticas de Spark
schema_dn_user = decode_schema(schema)

In [0]:
schema_dn_user

In [0]:
"""spark.sql(f"insert into hive_metastore.bronze.tbl_config_ingesta \
    select 1, 'keycloak', 'postgres', '', 'sql',\
    'bronze.dn_user', 'diaria', 1, current_timestamp(), '{str_schema}'")"""

 


In [0]:
# Leer la tabla "dn_user" desde PostgreSQL usando el esquema definido
# Para garantizar que los tipos de datos sean los correctos
df_dn_user = spark.read \
    .format("jdbc") \
    .option("url", jdbcUrl) \
    .option("dbtable", config["dbtable"]) \
    .option("user", connectionProperties["user"]) \
    .option("password", connectionProperties["password"]) \
    .option("driver", connectionProperties["driver"]) \
    .schema(schema_dn_user) \
    .load()

# Mostrar los primeros 10 registros para verificar la lectura
display(df_dn_user.limit(10))


In [0]:
#Validar el esquema
def validar_esquema_postgresql(jdbc_url, table_name, connection_props, schema_dn_user):

    # Leer la tabla desde PostgreSQL usando el esquema definido
    df_actual = spark.read.jdbc(url=jdbc_url, table=table_name, properties=connection_props)
    actual_schema = df_actual.schema

    # Comparar los esquemas
    expected_fields = {field.name: type(field.dataType) for field in schema_dn_user.fields}
    actual_fields = {field.name: type(field.dataType) for field in actual_schema.fields}

    errores = False # Variable para controlar si hay errores

    #Revisar si hay colummnas faltantes o con tipos diferentes
    for col in expected_fields:
        if col not in actual_fields:
            print(f"⚠️ Columna faltante: '{col}'")
            errores = True
        elif expected_fields[col] != actual_fields[col]:
            print(f"⚠️ Tipo cambiado en '{col}': esperado {expected_fields[col]}, recibido {actual_fields[col]}")
            errores = True

    #Revisar si hay columnas adicionales
    for col in actual_fields:
        if col not in expected_fields:
            print(f"⚠️ Columna adicional no esperada: '{col}'")
            errores = True

    # Mostrar el resdultado de si hubo o no errores
    if errores:
        print("❌ El esquema actual NO coincide con el esperado.")
    else:
        print("✅ El esquema actual coincide con el esperado.")

        # Mostrar los primeros 10 registros para verificar la lectura
        display(df_actual.limit(10))

# Ejecutar la validación
validar_esquema_postgresql(jdbcUrl, "dn_user", connectionProperties, schema_dn_user)

In [0]:
# Escribir en delta
df_dn_user.write.format("delta").mode("overwrite").saveAsTable(tabla_destino)
spark.sql(f"REFRESH TABLE {tabla_destino}")